In [1]:
import requests
import pandas as pd
import os
import akvut
kompisbrukar = os.environ["kompisbrukar"]
kompispassord = os.environ["kompispassord"]

logger = akvut.lag_logger("kompis_log")

# Hente data frå Outsystems (Kompis-prosjektet)
Eg autentiserer meg i utgangspunktet med brukar og passord fra environment-variabler, men den neste metoden er visstnok den sikraste, i følge Codeium. Den nyttar "auth=" frå requests.

Eg har ei liste med "org_id" som er interne nummer for kvar eining. Litt usikker på kvar eg finn dei, men eg har fått ei liste frå Håvard. Så leg eg alle data frå kvart slikt nummer.

In [2]:
# url = "https://hvl-dev.outsystemsenterprise.com/CV_API/rest/v1/"
# liste = [10013727, 10013737, 10013869, 10013870, 10013871, 10013872, 10013873, 10013874, 10013875, 10015026, 10013736]
liste = [10013736]
temp = []
for org_id in liste:
    url = f"https://hvl-dev.outsystemsenterprise.com/CV_API/rest/v1/HentAnsatte?Orgenhet={org_id}"
    # logger.info(f"Sender spørjing til {url}")
    resultat = requests.get(url, auth=(kompisbrukar, kompispassord))
    temp.append(pd.DataFrame(resultat.json()))
tilsette = pd.concat(df for df in temp if not df.empty).reset_index(drop=True)

In [3]:
tilsette

,navn,fodselsaar,ansattnr,utdanning,fakultet,nivaa3,utmerkelse,prosjekt,publikasjon,nivaa4,verv
0,Liv Kjelleberg,1976,03604715,Bachelorgrad,Fakultet for helse- og sosialvitskap,"FHS-Lederstøtte, HR, økonomi og FoU","[{'tittel': 'Medalje lang og tro tjeneste', 'a...",[{'tittel': 'Kunnskapsbasert praksis i videreu...,"[{'kategori': 'Vitenskapelig artikkel', 'titte...",NaN,NaN
1,Agnes Cecilie Wilhelmsen Giertsen,1958,03601175,Bachelorgrad,Fakultet for helse- og sosialvitskap,Institutt for helse- og omsorgsvitskap,NaN,"[{'tittel': 'Seksuell helsE-læring', 'rolle': ...","[{'kategori': 'Kronikk', 'tittel': 'Lev ditt e...",Master jordmor og helsesjukepleie,"[{'kategori': 'Forskargruppe', 'rolle': 'Leiar..."


## Plukke ut data
Så skal eg hente ut data. Det er kurant for alle felt utanom desse:

- utmerkelse (kva er dette?)
- prosjekt
- publikasjon
- verv

Alle desse er lister med eitt eller fleire element. Dersom eg berre ser på *eitt* av felta kan eg lage ei linje i tabellen for kvart element i lista, der alle andre er like. Alternativt må vi ha ein ekstern tabell for kvart felt, og i PowerBI hente ut data frå ulike databaser.

Eg hopper like godt rett i det: eg lager ein tabell for kvart felt som har fleire element.

### Lage nye tabellar
Eg kan ikkje opprette tabellar "automatisk", eg må gå gjennom samla data for å sjekke kva tabellar eg treng. Men i utgangspunktet treng eg dei fire som er nevnt ovanfor. Så eg lager like godt fire slike datarammer med ein gang:

In [23]:
tilsette.iloc[0]

navn                                              Liv Kjelleberg
fodselsaar                                                  1976
ansattnr                                                03604715
utdanning                                           Bachelorgrad
fakultet                    Fakultet for helse- og sosialvitskap
nivaa3                       FHS-Lederstøtte, HR, økonomi og FoU
utmerkelse     [{'tittel': 'Medalje lang og tro tjeneste', 'a...
prosjekt       [{'tittel': 'Kunnskapsbasert praksis i videreu...
publikasjon    [{'kategori': 'Vitenskapelig artikkel', 'titte...
nivaa4                                                       NaN
verv                                                         NaN
Name: 0, dtype: object

In [21]:
temp = []
for index, row in tilsette.iterrows():
    if isinstance(row['utmerkelse'], list):
        print(row['utmerkelse'])
        print(type(row['utmerkelse']))
        temp.append(pd.DataFrame(row['utmerkelse']))
utmerkelser = pd.concat(df for df in temp if not df.empty).reset_index(drop=True)

[{'tittel': 'Medalje lang og tro tjeneste', 'aar': '2020', 'utsteder': 'Høgskulen på Vestlandet', 'beskrivelse': 'sdklfjs'}, {'tittel': 'Test', 'aar': '2020', 'utsteder': 'Scollar ', 'land': 'Finland', 'beskrivelse': 'sdf'}]
<class 'list'>


In [22]:
utmerkelser

,tittel,aar,utsteder,beskrivelse,land
0,Medalje lang og tro tjeneste,2020,Høgskulen på Vestlandet,sdklfjs,NaN
1,Test,2020,Scollar,sdf,Finland


In [24]:
temp = []
for index, row in tilsette.iterrows():
    if isinstance(row['prosjekt'], list):
        print(row['prosjekt'])
        print(type(row['prosjekt']))
        temp.append(pd.DataFrame(row['prosjekt']))
prosjekt = pd.concat(df for df in temp if not df.empty).reset_index(drop=True)
prosjekt

[{'tittel': 'Kunnskapsbasert praksis i videreutdanning', 'rolle': 'Prosjektdeltaker', 'eier': 'Høgskulen på Vestlandet', 'fradato': '2009-10-05', 'tildato': '2020-01-02', 'totalbudsjett': '0', 'samarbeidspartner': [{'navn': 'Høgskulen på Vestlandet'}]}, {'tittel': 'Kunnskapsbasert praksis i utdanning', 'rolle': 'Prosjektdeltaker', 'eier': 'Høgskulen på Vestlandet', 'fradato': '2015-03-24', 'tildato': '2020-12-31', 'totalbudsjett': '0', 'kategori': [{'navn': 'Anvendt forskning'}, {'navn': 'Faglig utviklingsarbeid'}, {'navn': 'Doktorgradsprosjekt'}], 'samarbeidspartner': [{'navn': 'Høgskulen på Vestlandet'}, {'navn': 'Universitetet i Bergen'}]}, {'tittel': 'Kunnskapsbasert praksis i spesialisthelsetjenesten', 'rolle': 'Prosjektdeltaker', 'eier': 'Høgskulen på Vestlandet', 'fradato': '2015-03-24', 'tildato': '2020-12-31', 'totalbudsjett': '0', 'kategori': [{'navn': 'Anvendt forskning'}, {'navn': 'Doktorgradsprosjekt'}], 'samarbeidspartner': [{'navn': 'Høgskulen på Vestlandet'}]}, {'tittel

,tittel,rolle,eier,fradato,tildato,totalbudsjett,samarbeidspartner,kategori,finansieringskilde
0,Kunnskapsbasert praksis i videreutdanning,Prosjektdeltaker,Høgskulen på Vestlandet,2009-10-05,2020-01-02,0,[{'navn': 'Høgskulen på Vestlandet'}],NaN,NaN
1,Kunnskapsbasert praksis i utdanning,Prosjektdeltaker,Høgskulen på Vestlandet,2015-03-24,2020-12-31,0,"[{'navn': 'Høgskulen på Vestlandet'}, {'navn':...","[{'navn': 'Anvendt forskning'}, {'navn': 'Fagl...",NaN
2,Kunnskapsbasert praksis i spesialisthelsetjene...,Prosjektdeltaker,Høgskulen på Vestlandet,2015-03-24,2020-12-31,0,[{'navn': 'Høgskulen på Vestlandet'}],"[{'navn': 'Anvendt forskning'}, {'navn': 'Dokt...",NaN
3,Utvikling og implementering av nettbasert emne...,Prosjektleder,Høgskulen på Vestlandet,2018-05-07,2021-06-24,0,[{'navn': 'Høgskulen på Vestlandet'}],[{'navn': 'Internt prosjekt'}],NaN
4,Seksuell helsE-læring,Prosjektdeltaker,Norges teknisk-naturvitenskapelige universitet,2022-06-01,2024-12-31,1922000,[{'navn': 'Noregs teknisk-naturvitskaplege uni...,[{'navn': 'Anvendt forskning'}],Helsedirektoratet
5,Kartlegging av Ungdomsvennlighet i helsestasjo...,Prosjektdeltaker,Høgskulen på Vestlandet,2018-09-01,2021-06-30,0,[{'navn': 'Høgskulen på Vestlandet'}],[{'navn': 'Anvendt forskning'}],NaN


In [26]:
temp = []
for index, row in tilsette.iterrows():
    if isinstance(row['publikasjon'], list):
        print(row['publikasjon'])
        print(type(row['publikasjon']))
        temp.append(pd.DataFrame(row['publikasjon']))
publikasjoner = pd.concat(df for df in temp if not df.empty).reset_index(drop=True)
publikasjoner

[{'kategori': 'Vitenskapelig artikkel', 'tittel': 'Migrant Somali women’s experiences with their first contact with the labor ward prior to admission: A qualitative study', 'publiseringsaar': '2022', 'kanal': 'European Journal of Midwifery', 'forfatter': 'Eline Skirnisdottir Vik, Randa M. A. Hashi, Maryam Essa Hamud, Vigdis Aasheim, Tone Åslaug Kringeland, Katrine Aasekjær'}, {'kategori': 'Vitenskapelig Kapittel/Artikkel/Konferanseartikkel', 'tittel': 'Virtual Reality (VR) in Anatomy Teaching and Learning in Higher Healthcare Education', 'publiseringsaar': '2022', 'kanal': 'Springer', 'forfatter': 'Katrine Aasekjær, Beate Eltarvåg Gjesdal, Ivar Rosenberg, Lars Peder Vatshelle Bovim'}, {'kategori': 'Vitenskapelig artikkel', 'tittel': 'Collaborative learning in small groups in an online course – a case study', 'publiseringsaar': '2022', 'kanal': 'BMC Medical Education', 'nvinivaa': '1', 'forfatter': 'Mildrid Jorunn Haugland, Ivar Rosenberg, Katrine Aasekjær'}, {'kategori': 'Vitenskapelig

,kategori,tittel,publiseringsaar,kanal,forfatter,nvinivaa
0,Vitenskapelig artikkel,Migrant Somali women’s experiences with their ...,2022,European Journal of Midwifery,"Eline Skirnisdottir Vik, Randa M. A. Hashi, Ma...",NaN
1,Vitenskapelig Kapittel/Artikkel/Konferansearti...,Virtual Reality (VR) in Anatomy Teaching and L...,2022,Springer,"Katrine Aasekjær, Beate Eltarvåg Gjesdal, Ivar...",NaN
2,Vitenskapelig artikkel,Collaborative learning in small groups in an o...,2022,BMC Medical Education,"Mildrid Jorunn Haugland, Ivar Rosenberg, Katri...",1
3,Vitenskapelig artikkel,Midwives’ experiences with a safe childbirth c...,2023,Midwifery,"Eline Skirnisdottir Vik, Marte Østenfor Myre, ...",1
4,Vitenskapelig artikkel,Immersive Virtual Reality (VR) when learning a...,2024,European Journal of Midwifery,"Katrine Aasekjær, Bente Bjørnås, Halldis K. Sk...",NaN
5,Kronikk,Lev ditt eget liv før du begynner nytt liv,2022,Tidsskriftet sykepleien,"Merete Nypan, Agnes Cecilie Wilhelmsen Giertse...",NaN
6,Leserinnlegg,Lev ditt eget liv før du begynner et nytt liv,2022,Sykepleien.no,"Merete Nypan, Agnes Cecilie Wilhelmsen Giertse...",NaN
7,Leserinnlegg,Jenter får hjelp med kvinnehelse,2023,Tidsskriftet sykepleien,"Anne Holter Bentzrød, Merete Nypan, Agnes Ceci...",NaN
8,Faglig foredrag,Abstract -NACS conference 2023 OsloMet Septemb...,2023,NaN,"Camilla Rørtveit, Ann-Karin Valle, Maryam Rouh...",NaN
9,Leserinnlegg,Jenter får hjelp med kvinnehelse,2023,Sykepleien.no,"Anne Holter Bentzrød, Agnes Cecilie Wilhelmsen...",NaN
